In [1]:
%cd ..

c:\Users\HuyLQ15_CTV\Desktop\model_monitoring


In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression

data = load_breast_cancer(as_frame=True)['data']
data['target'] = load_breast_cancer(as_frame=True)['target']

X = data.drop('target', axis=1).iloc[:125]
y = data['target'].iloc[:125]
model = LogisticRegression()
model.fit(X, y)
prediction1 = model.predict(data.drop('target', axis=1))
prediction2 = model.predict_proba(data.drop('target', axis=1))
data['prediction'] = prediction1
for col, value in zip(model.classes_, prediction2.transpose()):
    data[col] = value
data.columns = data.columns.map(str)

reference = data.iloc[:200]
current = data.iloc[200:]

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.utils.data_operations import process_columns
from evidently.calculations.classification_performance import get_prediction_data
from source.metric_results.classification.confusion_matrix import calculate_confusion_matrix_by_classes
column_mapping = ColumnMapping()
column_mapping.target = 'target'
column_mapping.prediction = '1'
column_mapping.pos_label = 1

dataset_column = process_columns(data, column_mapping)
prediction_data = get_prediction_data(data, data_columns=dataset_column, pos_label=1, threshold = 0.5)

# Utils

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from source.metric_results.classification.confusion_matrix import calculate_confusion_by_classes

In [5]:
target = data[column_mapping.target]
prediction = prediction_data.predictions
prediction_probas = prediction_data.prediction_probas


In [6]:
accuracy = accuracy_score(target, prediction)
roc_auc = roc_auc_score(target, prediction_probas[column_mapping.pos_label], multi_class='ovr')
# accuracy
# log_loss(target, prediction_probas[column_mapping.pos_label])
classification_rp = classification_report(target, prediction, output_dict=True)
# classification_rp.pop('accuracy')
# calculate_confusion_matrix_by_classes(target, prediction)

# Custom Binary Classification Performance

In [7]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from source.metric_results.classification.classification_calculator import ClassificationResult
from source.metric_results.classification.classification_calculator import calculate_classification_results

calculate_classification_results(prediction, target, prediction_probas[column_mapping.pos_label])

ClassificationResult(type='evidently:metric_result:MultiClassificationResult', accuracy=0.9209138840070299, confusion_matrix={'0': {'tp': 207.0, 'tn': 317.0, 'fp': 40.0, 'fn': 5.0}, '1': {'tp': 317.0, 'tn': 207.0, 'fp': 5.0, 'fn': 40.0}}, classification_info={'0': {'precision': 0.8380566801619433, 'recall': 0.9764150943396226, 'f1-score': 0.9019607843137255, 'support': 212.0}, '1': {'precision': 0.984472049689441, 'recall': 0.8879551820728291, 'f1-score': 0.9337260677466863, 'support': 357.0}, 'macro avg': {'precision': 0.9112643649256922, 'recall': 0.9321851382062258, 'f1-score': 0.917843426030206, 'support': 569.0}, 'weighted avg': {'precision': 0.9299201018162784, 'recall': 0.9209138840070299, 'f1-score': 0.9218908479087466, 'support': 569.0}}, logloss=0.20927239133138018, roc_auc={'ovr': 0.9877517044553671, 'ovo': 0.9877517044553671}, mattheus_corr_coefficient=0.8431900056208114)

In [8]:
def get_classification_result(data: pd.DataFrame, column_mapping: ColumnMapping) -> ClassificationResult:
        
        dataset_column = process_columns(data, column_mapping)
        prediction_data = get_prediction_data(data, data_columns=dataset_column, pos_label=None, threshold=0.5)
        results = calculate_classification_results(
            prediction=prediction_data.predictions,
            target=data[dataset_column.utility_columns.target],
            prediction_proba=prediction_data.prediction_probas[column_mapping.pos_label]
        )
        return results
get_classification_result(current, column_mapping)

ClassificationResult(type='evidently:metric_result:MultiClassificationResult', accuracy=0.8970189701897019, confusion_matrix={'0': {'tp': 106.0, 'tn': 225.0, 'fp': 36.0, 'fn': 2.0}, '1': {'tp': 225.0, 'tn': 106.0, 'fp': 2.0, 'fn': 36.0}}, classification_info={'0': {'precision': 0.7464788732394366, 'recall': 0.9814814814814815, 'f1-score': 0.848, 'support': 108.0}, '1': {'precision': 0.9911894273127754, 'recall': 0.8620689655172413, 'f1-score': 0.9221311475409836, 'support': 261.0}, 'macro avg': {'precision': 0.868834150276106, 'recall': 0.9217752234993615, 'f1-score': 0.8850655737704918, 'support': 369.0}, 'weighted avg': {'precision': 0.9195668261205786, 'recall': 0.8970189701897019, 'f1-score': 0.9004342263094762, 'support': 369.0}}, logloss=0.2682398653708357, roc_auc={'ovr': 0.987264084007379, 'ovo': 0.987264084007379}, mattheus_corr_coefficient=0.7888348525943859)

In [9]:
import pandas as pd
from typing import Optional, Dict
from evidently.calculations.classification_performance import get_prediction_data
from evidently.renderers.base_renderer import MetricRenderer, default_renderer
from evidently.base_metric import InputData, Metric, MetricResult
from evidently.utils.data_operations import process_columns
from evidently import ColumnMapping
from evidently.base_metric import Metric
from source.metric_results.classification.classification_calculator import calculate_classification_results

class BinaryClassificationPerformanceResult(MetricResult):
    class Config:
        type_alias = "evidently:metric_result:BinaryClassificationPerformanceResult"
    reference: Optional[ClassificationResult]
    current: ClassificationResult

class BinaryClassificationPerformanceMetric(Metric[ClassificationResult]):
    class Config:
        type_alias = "evidently:metric:BinaryClassificationPerformanceMetric"
    _threshold: float
    def __init__(self, threshold: float = 0.5):
        self._threshold = threshold
        super().__init__()

    def calculate(self, data: InputData) -> ClassificationResult:
        results = {}
        results['reference'] = None
        if data.current_data is None:
            raise ValueError("The value cannot be None")

        if data.reference_data is not None:
            results['reference'] = self.get_classification_result(data.reference_data, data.column_mapping)

        results['current'] = self.get_classification_result(data.current_data, data.column_mapping)
        return BinaryClassificationPerformanceResult(
            reference=results['reference'],
            current=results['current']
        )
        
    def get_classification_result(self, data: pd.DataFrame, column_mapping: ColumnMapping) -> ClassificationResult:
        
        dataset_column = process_columns(data, column_mapping)
        prediction_data = get_prediction_data(data, data_columns=dataset_column, pos_label=None, threshold=self._threshold)
        results = calculate_classification_results(
            prediction=prediction_data.predictions,
            target=data[dataset_column.utility_columns.target],
            prediction_proba=prediction_data.prediction_probas[column_mapping.pos_label]
        )
        return results

In [1]:
from typing import List
from evidently.renderers.html_widgets import BaseWidgetInfo
from evidently.renderers.base_renderer import MetricRenderer, default_renderer


@default_renderer(wrap_type=BinaryClassificationPerformanceMetric)
class BinaryClassificationPerformanceRender(MetricRenderer):
    def render_json(self, obj: BinaryClassificationPerformanceMetric, include_render: bool = False,
        include: "IncludeOptions" = None, exclude: "IncludeOptions" = None,) -> dict:
        result = obj.get_result().get_dict(include_render, include, exclude)
        return result
    def render_html(self, obj: BinaryClassificationPerformanceMetric) -> List[BaseWidgetInfo]:
        pass

NameError: name 'default_renderer' is not defined

In [11]:
from evidently.report import Report

report = Report(metrics=[
    BinaryClassificationPerformanceMetric()
])

report.run(current_data=data, reference_data=None, column_mapping=column_mapping)
report.json()


'{"version": "0.4.39", "metrics": [{"metric": "BinaryClassificationPerformanceMetric", "result": {"reference": null, "current": {"accuracy": 0.9209138840070299, "confusion_matrix": {"0": {"tp": 207.0, "tn": 317.0, "fp": 40.0, "fn": 5.0}, "1": {"tp": 317.0, "tn": 207.0, "fp": 5.0, "fn": 40.0}}, "classification_info": {"0": {"precision": 0.8380566801619433, "recall": 0.9764150943396226, "f1-score": 0.9019607843137255, "support": 212.0}, "1": {"precision": 0.984472049689441, "recall": 0.8879551820728291, "f1-score": 0.9337260677466863, "support": 357.0}, "macro avg": {"precision": 0.9112643649256922, "recall": 0.9321851382062258, "f1-score": 0.917843426030206, "support": 569.0}, "weighted avg": {"precision": 0.9299201018162784, "recall": 0.9209138840070299, "f1-score": 0.9218908479087466, "support": 569.0}}, "logloss": 0.20927239133138018, "roc_auc": {"ovr": 0.9877517044553671, "ovo": 0.9877517044553671}, "mattheus_corr_coefficient": 0.8431900056208114}}}], "timestamp": "2024-11-20 11:48:

# Test

In [12]:
from evidently.report import Report
from source.metrics.classification_performance.binary_classification_performance import BinaryClassificationPerformanceMetric

report = Report(metrics=[
    BinaryClassificationPerformanceMetric()
])

report.run(current_data=data, reference_data=None, column_mapping=column_mapping)
report.json()


c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.Metric'>, 'evidently:metric:RegressionPerformanceMetrics') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.MetricResult'>, 'evidently:metric_result:BinaryClassificationPerformanceResult') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.Metric'>, 'evidently:metric:BinaryClassificationPerformanceMetric') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")


'{"version": "0.4.39", "metrics": [{"metric": "BinaryClassificationPerformanceMetric", "result": {"reference": null, "current": {"accuracy": 0.9209138840070299, "confusion_matrix": {"0": {"tp": 207.0, "tn": 317.0, "fp": 40.0, "fn": 5.0}, "1": {"tp": 317.0, "tn": 207.0, "fp": 5.0, "fn": 40.0}}, "classification_info": {"0": {"precision": 0.8380566801619433, "recall": 0.9764150943396226, "f1-score": 0.9019607843137255, "support": 212.0}, "1": {"precision": 0.984472049689441, "recall": 0.8879551820728291, "f1-score": 0.9337260677466863, "support": 357.0}, "macro avg": {"precision": 0.9112643649256922, "recall": 0.9321851382062258, "f1-score": 0.917843426030206, "support": 569.0}, "weighted avg": {"precision": 0.9299201018162784, "recall": 0.9209138840070299, "f1-score": 0.9218908479087466, "support": 569.0}}, "logloss": 0.20927239133138018, "roc_auc": {"ovr": 0.9877517044553671, "ovo": 0.9877517044553671}, "mattheus_corr_coefficient": 0.8431900056208114}}}], "timestamp": "2024-11-20 11:48: